In [1]:
push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main")
using DHC_2DUtils
using Profile
using BenchmarkTools
using FFTW
using HDF5
using Test
using SparseArrays
using Statistics
using Plots
using LinearAlgebra
using Measures
using Plots
using MLDatasets
using ImageCore
using Colors
using AbstractFFTs
using Interpolations
using DSP
using StatsBase
using Images

In [2]:
train_x, train_y = CIFAR10.traindata();
test_x, test_y   = CIFAR10.testdata();

In [3]:
function color_convert(array, color1, color2)
    chan_img = channelview(array)
    cv = colorview(color1, StackedView(array[:,:,1],array[:,:,2],array[:,:,3]))
    y_im = color2.(cv)
    channels = permutedims(channelview(float.(y_im)),(2,3,1))
    return channels
end

color_convert (generic function with 1 method)

In [4]:
filter_hash = fink_filter_hash(1, 8, nx=128, pc=1, wd=2)

function wind_2d(nx)
    dx   = nx/2-1
    filter = zeros(Float64, nx, nx)
    A = DSP.tukey(nx, 0.3)
    itp = extrapolate(interpolate(A,BSpline(Linear())),0)
    @inbounds for x = 1:nx
        sx = x-dx-1    # define sx,sy so that no fftshift() needed
        for y = 1:nx
            sy = y-dx-1
            r  = sqrt.((sx).^2 + (sy).^2) + nx/2
            filter[x,y] = itp(r)
        end
    end
    return filter
end

function wind_2d_RGB(nx)
    dx   = nx/2-1
    filter = zeros(Float64, nx, nx)
    A = DSP.tukey(nx, 0.3)
    itp = extrapolate(interpolate(A,BSpline(Linear())),0)
    @inbounds for x = 1:nx
        sx = x-dx-1    # define sx,sy so that no fftshift() needed
        for y = 1:nx
            sy = y-dx-1
            r  = sqrt.((sx).^2 + (sy).^2) + nx/2
            filter[x,y] = itp(r)
        end
    end
    return reshape(filter,nx,nx,1)
end

function cifar_pad_RGB(im; θ=0.0)
    imbig = convert(Array{Float64,3},imresize(im,(64,64,3)))
    datad_w = fweights(wind_2d(64));
    mu_imbig = zeros(1,1,3)
    for chan = 1:3
        mu_imbig[chan] = mean(imbig[:,:,chan],datad_w)
    end
    imbig .-= mu_imbig
    imbig .*= wind_2d_RGB(64)
    impad = zeros(Float64,128,128,3)
    impad[96:-1:33,33:96,:] = imbig

    if θ != 0.0
        imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
        imrot[findall(imrot .!= imrot)] .= 0.0
        return imrot .+ mu_imbig
    end

    return impad.+ mu_imbig
end

function cifar_DHC(x)
    image = cifar_pad_RGB(x[:,:,:], θ=0.0)
    WST = DHC_compute_RGB(image, filter_hash)
    return WST
end

cifar_DHC (generic function with 1 method)

In [10]:
cifar_DHC(train_x[:,:,:,10])

15258-element Array{Any,1}:
 0.24216718117732908
 0.22852803745142802
 0.2044979588366665
 0.005399006409819963
 0.005226048073279262
 0.0056879394648188366
 0.0007695632474127447
 0.011633073404047968
 0.04847540762629337
 0.0838663526338885
 0.06506477687209096
 0.0006585391185012399
 0.008877509566882638
 ⋮
 0.03381773618472951
 0.0019071496135401348
 0.01000322871212987
 0.02363731701077009
 0.028242399942008636
 0.04614659453399955
 0.0026156671561036984
 0.013802281528030534
 0.031152234447061868
 0.04118541951962858
 0.05644625521276226
 0.07860905328088953

In [5]:
using Distributed
using ProgressMeter

In [6]:
addprocs(7)

7-element Array{Int64,1}:
 2
 3
 4
 5
 6
 7
 8

In [7]:
lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x)[4]
    push!(lst_train,train_x[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x)[4]
    push!(lst_test,test_x[:,:,:,i])
end

In [16]:
@everywhere begin    
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main")
    using DHC_2DUtils
    using Profile
    using BenchmarkTools
    using FFTW
    using HDF5
    using Test
    using SparseArrays
    using Statistics
    using Plots
    using LinearAlgebra
    using Measures
    using Plots
    using MLDatasets
    using ImageCore
    using Colors
    using AbstractFFTs
    using Interpolations
    using DSP
    using StatsBase
    using Distributed
    using Images
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1, 8, nx=128, pc=1, wd=2)

    function wind_2d(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return filter
    end

    function wind_2d_RGB(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return reshape(filter,nx,nx,1)
    end

    function cifar_pad_RGB(im; θ=0.0)
        imbig = convert(Array{Float64,3},imresize(im,(64,64,3)))
        datad_w = fweights(wind_2d(64));
        mu_imbig = zeros(1,1,3)
        for chan = 1:3
            mu_imbig[chan] = mean(imbig[:,:,chan],datad_w)
        end
        imbig .-= mu_imbig
        imbig .*= wind_2d_RGB(64)
        impad = zeros(Float64,128,128,3)
        impad[96:-1:33,33:96,:] = imbig

        if θ != 0.0
            imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot .+ mu_imbig
        end

        return impad.+ mu_imbig
    end

    function cifar_DHC(x)
        image = cifar_pad_RGB(x[:,:,:], θ=0.0)
        WST = DHC_compute_RGB(image, filter_hash)
        return WST
    end
end
 
cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("../scratch_AKS/data/cifar10_train_RGB.h5", "main/train_data", cifar_DHC_out, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:40:54m:55


In [18]:
h5write("../scratch_AKS/data/cifar10_train_RGB.h5", "main/train_labels", train_y, deflate=3)

In [19]:
h5write("../scratch_AKS/data/cifar10_train_RGB.h5", "main/test_labels", test_y, deflate=3)

In [8]:
@everywhere begin    
    push!(LOAD_PATH, "/Users/saydjari/Dropbox/GradSchool_AKS/Doug/Projects/DHC/main")
    using DHC_2DUtils
    using Profile
    using BenchmarkTools
    using FFTW
    using HDF5
    using Test
    using SparseArrays
    using Statistics
    using Plots
    using LinearAlgebra
    using Measures
    using Plots
    using MLDatasets
    using ImageCore
    using Colors
    using AbstractFFTs
    using Interpolations
    using DSP
    using StatsBase
    using Distributed
    using Images
    using ProgressMeter
    
    # filter bank
    filter_hash = fink_filter_hash(1, 8, nx=128, pc=1, wd=2)

    function wind_2d(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return filter
    end

    function wind_2d_RGB(nx)
        dx   = nx/2-1
        filter = zeros(Float64, nx, nx)
        A = DSP.tukey(nx, 0.3)
        itp = extrapolate(interpolate(A,BSpline(Linear())),0)
        @inbounds for x = 1:nx
            sx = x-dx-1    # define sx,sy so that no fftshift() needed
            for y = 1:nx
                sy = y-dx-1
                r  = sqrt.((sx).^2 + (sy).^2) + nx/2
                filter[x,y] = itp(r)
            end
        end
        return reshape(filter,nx,nx,1)
    end

    function cifar_pad_RGB(im; θ=0.0)
        imbig = convert(Array{Float64,3},imresize(im,(64,64,3)))
        datad_w = fweights(wind_2d(64));
        mu_imbig = zeros(1,1,3)
        for chan = 1:3
            mu_imbig[chan] = mean(imbig[:,:,chan],datad_w)
        end
        imbig .-= mu_imbig
        imbig .*= wind_2d_RGB(64)
        impad = zeros(Float64,128,128,3)
        impad[96:-1:33,33:96,:] = imbig

        if θ != 0.0
            imrot = imrotate(impad, θ, axes(impad), Cubic(Throw(OnGrid())))
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot .+ mu_imbig
        end

        return impad.+ mu_imbig
    end

    function cifar_DHC(x)
        image = cifar_pad_RGB(x[:,:,:], θ=0.0)
        WST = DHC_compute_RGB(image, filter_hash)
        return WST
    end
end

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("../scratch_AKS/data/cifar10_train_RGB.h5", "main/test_data", cifar_DHC_out, deflate=3)

Progress: 100%|█████████████████████████████████████████| Time: 0:06:226:56


In [34]:
function transformMaker2(coeff, S1Mat, S2Mat; Nc=1)
    NS1 = size(S1Mat)[2]
    NS2 = size(S2Mat)[2]
    if Nc==1
        S0iso = coeff[:,1:2]
        S1iso = transpose(S1Mat*transpose(coeff[:,2+1:2+NS1]))
        S2iso = transpose(S2Mat*transpose(coeff[:,2+NS1+1:2+NS1+NS2]))
    else
        S0iso = coeff[:,1:2*Nc]
        S1MatChan = blockdiag(collect(Iterators.repeated(S1Mat,Nc))...)
        S2MatChan = blockdiag(collect(Iterators.repeated(S2Mat,Nc*Nc))...)
        S1iso = transpose(S1MatChan*transpose(coeff[:,2*Nc+1:2*Nc+Nc*NS1]))
        S2iso = transpose(S2MatChan*transpose(coeff[:,2*Nc+Nc*NS1+1:end]))
    end
    return hcat(S0iso,S1iso,S2iso)
end

transformMaker2 (generic function with 2 methods)

In [12]:
size(cifar_DHC_out')

(10000, 15258)

In [35]:
cifar_DHC_out_iso = transformMaker2(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

10000×1923 Array{Float64,2}:
 0.455073  0.447771  0.422572  0.00454728  …  0.235451  0.243422   0.0629224
 0.585832  0.600157  0.646387  0.00982033     0.33834   0.494112   0.129739
 0.491091  0.543293  0.587751  0.00474712     0.355125  0.308975   0.0835853
 0.626379  0.608432  0.663941  0.00421128     0.189571  0.234432   0.0455583
 0.467555  0.499154  0.388241  0.00332022     0.29489   0.350986   0.0733963
 0.532734  0.359601  0.283212  0.0046083   …  0.241347  0.26346    0.0529321
 0.439984  0.308048  0.309626  0.00614657     0.241331  0.268009   0.0535944
 0.403258  0.42991   0.247556  0.00207121     0.298033  0.317753   0.0640879
 0.704432  0.658716  0.606829  0.00157091     0.073418  0.0710014  0.00896428
 0.458166  0.450981  0.573334  0.00491028     0.319556  0.417086   0.100434
 0.520875  0.584695  0.616177  0.00304151  …  0.114456  0.0977631  0.0165017
 0.490225  0.467566  0.47439   0.00618569     0.3021    0.373562   0.0990603
 0.483584  0.437728  0.358997  0.00504391     0.

In [36]:
h5write("../scratch_AKS/data/cifar10_RGB.h5", "main/test_data_iso", cifar_DHC_out_iso, deflate=3)

In [37]:
cifar_DHC_out = h5read("../scratch_AKS/data/cifar10_RGB.h5", "main/train_data")

15258×50000 Array{Float64,2}:
 0.580269     0.564318    0.46359      …  0.333831     0.657957    0.491289
 0.436284     0.5634      0.458437        0.416181     0.687953    0.44982
 0.286279     0.56411     0.449199        0.462129     0.728403    0.438073
 0.00364376   0.00715925  0.0110784       0.00739665   0.0039513   0.0039323
 0.00446846   0.00644805  0.0112745       0.00887945   0.00361248  0.00487962
 0.00444399   0.00597261  0.0105201    …  0.0107721    0.00422244  0.00516922
 0.00117244   0.00344843  0.000456295     0.000938051  0.00538834  0.00378628
 0.0105963    0.0283759   0.0057753       0.011769     0.033561    0.0387692
 0.0191871    0.0600018   0.0133893       0.0352777    0.0782995   0.0858004
 0.0852615    0.116535    0.0403329       0.0466474    0.128305    0.0881797
 0.0424589    0.0296948   0.0967231    …  0.0891882    0.0268126   0.0261332
 0.000969586  0.00257553  0.000342689     0.000722373  0.00336388  0.00324193
 0.0103257    0.0210314   0.00531722      0.00

In [38]:
cifar_DHC_out_iso = transformMaker2(cifar_DHC_out',filter_hash["S1_iso_mat"],filter_hash["S2_iso_mat"],Nc=3)

50000×1923 Array{Float64,2}:
 0.580269  0.436284  0.286279  …  0.14994   0.120892   0.01762
 0.564318  0.5634    0.56411      0.184861  0.144118   0.0295423
 0.46359   0.458437  0.449199     0.33072   0.424569   0.102412
 0.441879  0.36146   0.251306     0.130787  0.0815844  0.00899749
 0.359007  0.400299  0.452498     0.160675  0.106516   0.0256936
 0.569576  0.261594  0.224844  …  0.288434  0.378896   0.087813
 0.45265   0.585911  0.340866     0.138268  0.119457   0.0203213
 0.530125  0.490289  0.359276     0.20328   0.138219   0.0249112
 0.424513  0.598856  0.720087     0.145608  0.113481   0.0193942
 0.242167  0.228528  0.204498     0.266761  0.335046   0.0786091
 0.327895  0.339138  0.256802  …  0.154478  0.156217   0.0258254
 0.560814  0.494198  0.466773     0.29786   0.360036   0.0795406
 0.444829  0.496809  0.476855     0.319207  0.402469   0.0933523
 ⋮                             ⋱  ⋮                    
 0.640423  0.585768  0.516971     0.217314  0.229026   0.0488453
 0.60311

In [39]:
h5write("../scratch_AKS/data/cifar10_RGB.h5", "main/train_data_iso", cifar_DHC_out_iso, deflate=3)

In [ ]:
train_x_ycbcr = zeros(size(train_x))
for i=1:size(train_x)[4]
    train_x_ycbcr[:,:,:,i] = color_convert(train_x[:,:,:,i],RGB,YCbCr)
end
test_x_ycbcr = zeros(size(test_x))
for i=1:size(test_x)[4]
    test_x_ycbcr[:,:,:,i] = color_convert(test_x[:,:,:,i],RGB,YCbCr)
end

lst_train = Array{Any}(undef, 0)
for i = 1:size(train_x_ycbcr)[4]
    push!(lst_train,train_x_ycbcr[:,:,:,i])
end

lst_test = Array{Any}(undef, 0)
for i = 1:size(test_x_ycbcr)[4]
    push!(lst_test,test_x_ycbcr[:,:,:,i])
end

In [ ]:
h5write("../scratch_AKS/data/cifar10_YCbCr.h5", "main/test_labels", test_y, deflate=3)
h5write("../scratch_AKS/data/cifar10_YCbCr.h5", "main/train_labels", train_y, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_train)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("../scratch_AKS/data/cifar10_YCbCr.h5", "main/train_data", cifar_DHC_out, deflate=3)

cifar_DHC_out = @showprogress pmap(cifar_DHC, lst_test)
cifar_DHC_out = hcat(cifar_DHC_out...)

h5write("../scratch_AKS/data/cifar10_YCbCr.h5", "main/test_data", cifar_DHC_out, deflate=3)